In [1]:
import xml
import pandas as pd
import xml.etree.ElementTree as ET
import numpy as np

In [3]:
# name = "Animation_720P-0acc.csv"
name = "Animation_1080P-3d67.csv"

var = ['cabac', 'ref', 'subme', 'mixed_ref', 'me_range', 'trellis', '8x8dct', 'fast_pskip', 'chroma_qp_offset',
       'bframes', 'b_pyramid', 'b_adapt', 'weightb', 'open_gop', 'weightp', 'scenecut', 'rc_lookahead', 
       'mbtree', 'qpmax', 'aq-mode', 'size']

data = pd.read_csv(name,index_col=0)[var]
sizes = np.array(data["size"])
std_sizes = [(sizes[k]-np.mean(sizes))/np.std(sizes) for k in range(len(sizes))]
df = pd.get_dummies(data, columns=data.columns[:-1])

# data["size"]

names = df.columns

vm = ET.Element("vm", name = name[:-4])
bin_opt = ET.SubElement(vm, "binaryOptions")

for nbo in names:
    inter = ET.SubElement(bin_opt, "configurationOption")
    ET.SubElement(inter, "name").text = nbo
    ET.SubElement(inter, "optional").text = "True"

tree = ET.ElementTree(vm)
tree.write("fm"+name[:-4]+".xml")

vm = ET.Element("results")


for i in range(df.shape[0]):
    num_opt = ET.SubElement(vm, "row")
    inter = ET.SubElement(num_opt, "data")
    inter.set("columnname","Configuration")
    final_config=""
    for col in df.columns:
        arr = np.copy(df[col])
        if col != "size" and arr[i]:
            final_config+=col+','
    inter.text = final_config
    inter = ET.SubElement(num_opt, "data")
    inter.set("columnname","size")
    inter.text = str(std_sizes[i])
    

tree = ET.ElementTree(vm)
tree.write("measurement_"+name[:-4]+".xml")